Tugas mid test course NLP

Tawang sahro winanto

NLP machine learning

In [ ]:
pip install sastrawi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 209 kB 5.1 MB/s 


In [ ]:
import nltk
import pandas as pd
import re
import tensorflow as tf
import time
import multiprocessing
import io
import gensim
import numpy as np
import matplotlib.pyplot as plt

import pickle
import keras
import nltk

from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam
#from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Embedding
from keras.layers import Dense, Activation, Embedding, LSTM, Bidirectional, Dropout, GRU
from keras import regularizers
from tensorflow.keras.utils import to_categorical
from keras.models import load_model
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn import metrics
from nltk.tokenize import TweetTokenizer
from collections import defaultdict
from datetime import timedelta
from gensim.models import word2vec
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer


nltk.download('punkt')
from nltk.tokenize import word_tokenize

from google.colab import drive
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df = pd.read_csv("/content/drive/My Drive/MidTest Tawang sahro winanto/review_genshin.csv")
df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,6fb139b8-65ba-4a72-9b34-52791072cd01,Big L Gamings,https://play-lh.googleusercontent.com/a-/AFdZu...,Untuk anniv kedepanya semoga tidak seperti yan...,1,0,3.0.0_9612129_9624836,2022-09-14 08:54:25,NaN,NaN
1,a6812bc1-ab78-425a-b974-0e2ede67fe8a,Lxyna Shouguns,https://play-lh.googleusercontent.com/a-/AFdZu...,developer tolong perbaiki bug darah musuh yang...,1,0,3.0.0_9612129_9624836,2022-09-13 03:54:53,NaN,NaN
2,5a0558c6-eb98-492a-b976-a99a95a63a6f,nurul auliyah askar,https://play-lh.googleusercontent.com/a-/AFdZu...,Aku sukaaa bangeet tapi saat mau lanjutin yang...,1,7,3.0.0_9612129_9624836,2022-08-29 08:24:49,"Halo, Traveler! Terima kasih atas dukungan Tra...",2022-09-14 15:50:49
3,e47e01a0-e70e-4457-a49f-c2b01fa36a6e,Fuzzy Academy,https://play-lh.googleusercontent.com/a-/AFdZu...,Bisakah saya tidak mendapatkan b5 karakter di ...,1,14,3.0.0_9612129_9624836,2022-08-27 03:50:46,"Halo, Traveler! Traveler dapat melihat aturan ...",2022-09-14 15:24:15
4,deda26ff-203d-4506-afb6-d65e4892d6bb,jhonson pratama,https://play-lh.googleusercontent.com/a/AItbvm...,"Jaringan lelet sekali,selalu aja (menghubungka...",1,2,3.0.0_9612129_9624836,2022-09-04 21:30:17,NaN,NaN


In [ ]:
#menampilkan data rating 1
df[df['score'] < 5]

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,6fb139b8-65ba-4a72-9b34-52791072cd01,Big L Gamings,https://play-lh.googleusercontent.com/a-/AFdZu...,Untuk anniv kedepanya semoga tidak seperti yan...,1,0,3.0.0_9612129_9624836,2022-09-14 08:54:25,NaN,NaN
1,a6812bc1-ab78-425a-b974-0e2ede67fe8a,Lxyna Shouguns,https://play-lh.googleusercontent.com/a-/AFdZu...,developer tolong perbaiki bug darah musuh yang...,1,0,3.0.0_9612129_9624836,2022-09-13 03:54:53,NaN,NaN
2,5a0558c6-eb98-492a-b976-a99a95a63a6f,nurul auliyah askar,https://play-lh.googleusercontent.com/a-/AFdZu...,Aku sukaaa bangeet tapi saat mau lanjutin yang...,1,7,3.0.0_9612129_9624836,2022-08-29 08:24:49,"Halo, Traveler! Terima kasih atas dukungan Tra...",2022-09-14 15:50:49
3,e47e01a0-e70e-4457-a49f-c2b01fa36a6e,Fuzzy Academy,https://play-lh.googleusercontent.com/a-/AFdZu...,Bisakah saya tidak mendapatkan b5 karakter di ...,1,14,3.0.0_9612129_9624836,2022-08-27 03:50:46,"Halo, Traveler! Traveler dapat melihat aturan ...",2022-09-14 15:24:15
4,deda26ff-203d-4506-afb6-d65e4892d6bb,jhonson pratama,https://play-lh.googleusercontent.com/a/AItbvm...,"Jaringan lelet sekali,selalu aja (menghubungka...",1,2,3.0.0_9612129_9624836,2022-09-04 21:30:17,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
1995,e75aab24-1941-4920-82d6-544b57c6a903,Clarissa Risma,https://play-lh.googleusercontent.com/a-/AFdZu...,Mau download kok ngga bs y. Penyimpanan gua si...,1,0,NaN,2022-01-13 00:53:45,Kami mohon maaf atas ketidaknyamanan yang Anda...,2022-01-15 07:19:28
1996,51920e4d-2a6f-405b-876f-d62c32cfc731,DAVA A.R,https://play-lh.googleusercontent.com/a-/AFdZu...,Untuk Sementara Saya Kasih Bintang Satu Karna ...,1,6,2.4.0_5199000_5224067,2022-01-05 04:55:43,NaN,NaN
1997,41e867d5-4e95-41f3-9370-735155762a9f,manusia haram,https://play-lh.googleusercontent.com/a-/AFdZu...,"Game nya berat banget,kayak cinta kita yang be...",1,0,NaN,2022-06-06 22:25:13,Kami mohon maaf atas ketidaknyamanan yang Anda...,2022-06-15 10:43:59
1998,496e297c-62a9-4c0f-8930-54b36b48ed8b,Adzani N.j,https://play-lh.googleusercontent.com/a/AItbvm...,"Kenapa saya tidak bisa masuk ke game ini,game ...",1,2,2.3.0_4786731_4861639,2021-12-13 05:45:20,Kami mohon maaf atas ketidaknyamanan yang Anda...,2021-12-15 02:48:09


In [ ]:
#menampilkan data rating 5
df[df['score'] >1]

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
2000,58168b16-9a7b-40d9-85fb-1b0f48fddfb8,Dinda Naafi,https://play-lh.googleusercontent.com/a/AItbvm...,"Ini game grafiknya bagus, alur ceritanya juga ...",5,65,3.0.0_9612129_9624836,2022-09-08 08:19:02,NaN,NaN
2001,15474d2b-eff3-4490-a858-125aa28ad4d4,Nurdiansyah Putra S.,https://play-lh.googleusercontent.com/a-/AFdZu...,Dari segi UI menu mudah d pahami & tidak bikin...,5,443,3.0.0_9612129_9624836,2022-08-24 00:20:39,NaN,NaN
2002,cfcdfe87-7fc5-4572-97ec-0a3531628a08,Wilman Abisiena,https://play-lh.googleusercontent.com/a/AItbvm...,Mungkin masih menjadi mobile game dgn tema ope...,5,272,3.0.0_9612129_9624836,2022-08-25 01:57:44,NaN,NaN
2003,b1ebfca5-8d11-4178-b5e9-ddfce7e60365,Haikal Fasha,https://play-lh.googleusercontent.com/a-/AFdZu...,"Overall gamenya bagus, dari story gamenya, mek...",5,169,3.0.0_9612129_9624836,2022-08-26 15:33:09,NaN,NaN
2004,eb3c229b-9f2d-404f-ac03-2d6aabd96080,Hilman Hilman,https://play-lh.googleusercontent.com/a-/AFdZu...,Hallo saya punya saran dari saya untuk memuask...,5,15,3.0.0_9612129_9624836,2022-09-09 02:38:22,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
3995,5b434707-9d50-4429-903f-0a371370508d,Alkindi Mulia Achmad,https://play-lh.googleusercontent.com/a-/AFdZu...,Bagi yang bilang kalo dapat karakter yang sama...,5,1,1.2.0_1565149_1627898,2021-01-25 16:10:22,NaN,NaN
3996,00ab8821-abad-4b2a-b64c-2d798bd37c9b,Nafhisya prisilla Santoso,https://play-lh.googleusercontent.com/a-/AFdZu...,Game nya seru bngt grafik nya juga bagus,5,0,2.8.0_7997424_8078355,2022-08-24 05:38:36,NaN,NaN
3997,516ab213-8acb-4ac5-98ad-b5b6c0f8a71c,Susilo Susilo,https://play-lh.googleusercontent.com/a/AItbvm...,"Hallo pihak hoyoverse Indonesia, saya sangat m...",5,0,NaN,2022-08-23 02:39:03,NaN,NaN
3998,00d2a994-9c9d-4c6d-95c2-36adde3e91c6,Resha Palepi,https://play-lh.googleusercontent.com/a-/AFdZu...,"Penuh dengan updatean berat, tp seru banget",5,0,2.7.0_6933856_6989239,2022-07-11 08:10:27,NaN,NaN


In [ ]:
# menampilkan data hanya data username ,score,content
df[['userName', 'score', 'content']]

,userName,score,content
0,Big L Gamings,1,Untuk anniv kedepanya semoga tidak seperti yan...
1,Lxyna Shouguns,1,developer tolong perbaiki bug darah musuh yang...
2,nurul auliyah askar,1,Aku sukaaa bangeet tapi saat mau lanjutin yang...
3,Fuzzy Academy,1,Bisakah saya tidak mendapatkan b5 karakter di ...
4,jhonson pratama,1,"Jaringan lelet sekali,selalu aja (menghubungka..."
...,...,...,...
3995,Alkindi Mulia Achmad,5,Bagi yang bilang kalo dapat karakter yang sama...
3996,Nafhisya prisilla Santoso,5,Game nya seru bngt grafik nya juga bagus
3997,Susilo Susilo,5,"Hallo pihak hoyoverse Indonesia, saya sangat m..."
3998,Resha Palepi,5,"Penuh dengan updatean berat, tp seru banget"





---


Tahap preprocesing data

  1.case folding

  2.filtering

  3.tokenization

  4.Konversi Slangword

  5.Stopword Removal

  6.Stemming


---



1.casefolding

In [ ]:
dfc = df['content']
print(dfc[:4])

0    Untuk anniv kedepanya semoga tidak seperti yan...
1    developer tolong perbaiki bug darah musuh yang...
2    Aku sukaaa bangeet tapi saat mau lanjutin yang...
3    Bisakah saya tidak mendapatkan b5 karakter di ...
Name: content, dtype: object


In [ ]:
#casefolding
dfcf = dfc.str.lower()
dfcf.head()

0    untuk anniv kedepanya semoga tidak seperti yan...
1    developer tolong perbaiki bug darah musuh yang...
2    aku sukaaa bangeet tapi saat mau lanjutin yang...
3    bisakah saya tidak mendapatkan b5 karakter di ...
4    jaringan lelet sekali,selalu aja (menghubungka...
Name: content, dtype: object

2.filtering

In [ ]:
#filtering

#url
filtering_url = [re.sub(r'''(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|
\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'".,<>?«»“”‘’]))''',
 " ", tweet) for tweet in dfcf]
#cont
filtering_cont = [re.sub(r'\(cont\)'," ", tweet)for tweet in filtering_url]
#punctuatuion
filtering_punctuation = [re.sub('[!"”#$%&’()*+,-./:;<=>?@[\]^_`{|}~]', ' ', tweet) for tweet in filtering_cont]
#hapus simbol'[!#?,.:";@()-_/\']'
#  hapus #tagger
filtering_tagger = [re.sub(r'#([^\s]+)', '', tweet) for tweet in filtering_punctuation]
#numeric
filtering_numeric = [re.sub(r'\d+', ' ', tweet) for tweet in filtering_tagger]

# # filtering RT , @ dan #
# fungsi_clen_rt = lambda x: re.compile('\#').sub('', re.compile('rt @').sub('@', x, count=1).strip())
# clean = [fungsi_clen_rt for tweet in filtering_numeric]

data_filtering = pd.Series(filtering_numeric)


In [ ]:
data_filtering

0       untuk anniv kedepanya semoga tidak seperti yan...
1       developer tolong perbaiki bug darah musuh yang...
2       aku sukaaa bangeet tapi saat mau lanjutin yang...
3       bisakah saya tidak mendapatkan b  karakter di ...
4       jaringan lelet sekali selalu aja  menghubungka...
                              ...                        
3995    bagi yang bilang kalo dapat karakter yang sama...
3996             game nya seru bngt grafik nya juga bagus
3997    hallo pihak hoyoverse indonesia  saya sangat m...
3998          penuh dengan updatean berat  tp seru banget
3999    game nya keren aku sangat suka     dan bisa me...
Length: 4000, dtype: object

3.tokenization

In [ ]:
#tokenization
data_tokens = [word_tokenize(line) for line in data_filtering]
print(data_tokens)

[['untuk', 'anniv', 'kedepanya', 'semoga', 'tidak', 'seperti', 'yang', 'dulu', 'tolong', 'situ', 'beri', 'apresiasi', 'ke', 'player', 'bukan', 'cuman', 'player', 'aja', 'yang', 'ngasih', 'apresiasi', 'ke', 'situ', 'kalau', 'memuaskan', 'nanti', 'saya', 'tambahin', 'bintang', 'lagi'], ['developer', 'tolong', 'perbaiki', 'bug', 'darah', 'musuh', 'yang', 'terkadang', 'suka', 'hilang', 'itu', 'membuat', 'saya', 'kesulitan', 'untuk', 'farming', 'jika', 'sudah', 'di', 'perbaiki', 'akan', 'saya', 'beri', 'bintang', 'lagi'], ['aku', 'sukaaa', 'bangeet', 'tapi', 'saat', 'mau', 'lanjutin', 'yang', 'request', 'kembang', 'api', 'yang', 'pertama', 'kali', 'travelers', 'masuk', 'inazuma', 'itu', 'gw', 'kaga', 'bisa', 'masuk', 'ruang', 'bawah', 'tanah', 'loding', 'mulu', 'dari', 'kemarin', 'padahal', 'jaringan', 'bagus', 'aku', 'sudah', 'stel', 'stel', 'tapi', 'kagak', 'guna', 'dowload', 'apk', 'membantu', 'kagak', 'guna', 'bersihkan', 'ruang', 'juga', 'gak', 'guna', 'hapus', 'data', 'change', 'kagak

4.Konversi Slangword

In [ ]:
#slang word
path_dataslang = open("/content/drive/My Drive/MidTest Tawang sahro winanto/kamus kata baku.csv")
dataslang = pd.read_csv(path_dataslang, encoding = 'utf-8', header=None, sep=";")

def replaceSlang(word):
  if word in list(dataslang[0]):
    indexslang = list(dataslang[0]).index(word)
    return dataslang[1][indexslang]
  else:
    return word

data_formal = []
for data in data_tokens:
  data_clean = [replaceSlang(word) for word in data]
  data_formal.append(data_clean)
len_data_formal = len(data_formal)
print(data_formal)
len_data_formal

[['untuk', 'anniv', 'kedepanya', 'semoga', 'tidak', 'seperti', 'yang', 'dulu', 'tolong', 'situ', 'beri', 'apresiasi', 'ke', 'player', 'bukan', 'cuma', 'player', 'saja', 'yang', 'memberi', 'apresiasi', 'ke', 'situ', 'kalau', 'memuaskan', 'nanti', 'saya', 'tambahkan', 'bintang', 'lagi'], ['developer', 'tolong', 'perbaiki', 'bug', 'darah', 'musuh', 'yang', 'terkadang', 'suka', 'hilang', 'itu', 'membuat', 'saya', 'kesulitan', 'untuk', 'farming', 'jika', 'sudah', 'di', 'perbaiki', 'akan', 'saya', 'beri', 'bintang', 'lagi'], ['aku', 'suka', 'abnget', 'tapi', 'saat', 'mau', 'lanjutkan', 'yang', 'permintaan', 'kembang', 'api', 'yang', 'pertama', 'kali', 'travelers', 'masuk', 'inazuma', 'itu', 'saya', 'tidak', 'bisa', 'masuk', 'ruang', 'bawah', 'tanah', 'loding', 'terus', 'dari', 'kemarin', 'padahal', 'jaringan', 'bagus', 'aku', 'sudah', 'stel', 'stel', 'tapi', 'tidak', 'guna', 'dowload', 'apk', 'membantu', 'tidak', 'guna', 'bersihkan', 'ruang', 'juga', 'tidak', 'guna', 'hapus', 'data', 'change

4000

5.Stopword Removal

In [ ]:
nltk.download('stopwords')
default_stop_words = nltk.corpus.stopwords.words('indonesian')
stopwords = set(default_stop_words)

def removeStopWords(line, stopwords):
  words = []
  for word in line:
    word=str(word)
    word = word.strip()
    if word not in stopwords and word != "" and word != "&":
      words.append(word)

  return words
data_notstopword = [removeStopWords(line,stopwords) for line in data_formal]
len(data_notstopword)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


4000

In [ ]:
data_notstopword

[['anniv',
  'kedepanya',
  'semoga',
  'tolong',
  'situ',
  'apresiasi',
  'player',
  'player',
  'apresiasi',
  'situ',
  'memuaskan',
  'tambahkan',
  'bintang'],
 ['developer',
  'tolong',
  'perbaiki',
  'bug',
  'darah',
  'musuh',
  'terkadang',
  'suka',
  'hilang',
  'kesulitan',
  'farming',
  'perbaiki',
  'bintang'],
 ['suka',
  'abnget',
  'lanjutkan',
  'permintaan',
  'kembang',
  'api',
  'kali',
  'travelers',
  'masuk',
  'inazuma',
  'masuk',
  'ruang',
  'tanah',
  'loding',
  'kemarin',
  'jaringan',
  'bagus',
  'stel',
  'stel',
  'dowload',
  'apk',
  'membantu',
  'bersihkan',
  'ruang',
  'hapus',
  'data',
  'change',
  'memperbaiki',
  'sistem',
  'genshin',
  'loding',
  'layarnya',
  'item',
  'masuk',
  'pintu',
  'rahasia'],
 ['b',
  'karakter',
  'pity',
  'merah',
  'b',
  'karakter',
  'kudapatkan',
  'pity',
  'merah',
  'berkontribusi',
  'top',
  'up',
  'teman',
  'mengeluarkan',
  'sepeserpun',
  'uang',
  'game',
  'menang',
  'rate',
  'off',

6.Stemming

In [ ]:
white_list = ["bali"] #ini perlu/tidak perlu diubah karena dianggap sastrawi sebagai imbuhan i

factory = StemmerFactory()
ind_stemmer = factory.create_stemmer()
def stemmer(line):
    temp = list()
    for word in line:
      if(word not in white_list):
        word = ind_stemmer.stem(word)
      if(len(word)>3):
        temp.append(word)
    return temp

reviews = [stemmer (line) for line in data_notstopword]
print(reviews)

[['anniv', 'depa', 'moga', 'tolong', 'situ', 'apresiasi', 'player', 'player', 'apresiasi', 'situ', 'muas', 'tambah', 'bintang'], ['developer', 'tolong', 'baik', 'darah', 'musuh', 'terkadang', 'suka', 'hilang', 'sulit', 'farming', 'baik', 'bintang'], ['suka', 'abnget', 'lanjut', 'minta', 'kembang', 'kali', 'travelers', 'masuk', 'inazuma', 'masuk', 'ruang', 'tanah', 'loding', 'kemarin', 'jaring', 'bagus', 'stel', 'stel', 'dowload', 'bantu', 'ruang', 'hapus', 'data', 'change', 'baik', 'sistem', 'genshin', 'loding', 'layar', 'item', 'masuk', 'pintu', 'rahasia'], ['karakter', 'pity', 'merah', 'karakter', 'dapat', 'pity', 'merah', 'kontribusi', 'teman', 'keluar', 'peser', 'uang', 'game', 'menang', 'rate', 'kali', 'kali', 'pity', 'kuning'], ['jaring', 'lambat', 'hubung', 'server', 'loading', 'ulang', 'jaring', 'internet', 'bagus', 'buka', 'youtube', 'lancar', 'main', 'genshin', 'muas', 'jaring', 'buruk', 'gamenya'], ['butuh', 'fitur', 'lewat', 'dialog', 'main', 'batas', 'tinggal', 'bebagai', 

Feature Extraction (TF-IDF)

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer=TfidfVectorizer ( )
reviews2 = [ " " .join ( r ) for r in reviews ]
vektor_tfidf = vectorizer.fit_transform ( reviews2 )
vektor_tfidf = vektor_tfidf.toarray ( )
vektor_tfidf.shape
print(vectorizer.vocabulary_)

{'anniv': 216, 'depa': 1103, 'moga': 3303, 'tolong': 5264, 'situ': 4748, 'apresiasi': 257, 'player': 4004, 'muas': 3349, 'tambah': 5030, 'bintang': 615, 'developer': 1128, 'baik': 419, 'darah': 1049, 'musuh': 3389, 'terkadang': 5159, 'suka': 4946, 'hilang': 2053, 'sulit': 4951, 'farming': 1550, 'abnget': 17, 'lanjut': 2785, 'minta': 3258, 'kembang': 2500, 'kali': 2377, 'travelers': 5304, 'masuk': 3073, 'inazuma': 2170, 'ruang': 4388, 'tanah': 5040, 'loding': 2902, 'kemarin': 2495, 'jaring': 2281, 'bagus': 408, 'stel': 4876, 'dowload': 1324, 'bantu': 453, 'hapus': 1998, 'data': 1058, 'change': 818, 'sistem': 4744, 'genshin': 1801, 'layar': 2815, 'item': 2248, 'pintu': 3985, 'rahasia': 4175, 'karakter': 2402, 'pity': 3989, 'merah': 3195, 'dapat': 1041, 'kontribusi': 2665, 'teman': 5114, 'keluar': 2486, 'peser': 3954, 'uang': 5368, 'game': 1691, 'menang': 3140, 'rate': 4205, 'kuning': 2713, 'lambat': 2765, 'hubung': 2110, 'server': 4651, 'loading': 2893, 'ulang': 5388, 'internet': 2221, '

In [ ]:
print(vektor_tfidf)
print(vectorizer.get_feature_names())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
['aaaa', 'aaaaa', 'aaaaaaa', 'aaahaaahahahaaa', 'aathdagr', 'abad', 'abadi', 'abah', 'abai', 'abbys', 'abbyss', 'abgret', 'abis', 'abisin', 'abisssss', 'abistu', 'able', 'abnget', 'about', 'abstrak', 'absurd', 'abyes', 'abys', 'abyss', 'abyssosque', 'abyys', 'acak', 'acakana', 'acara', 'accept', 'account', 'achievement', 'acquaint', 'acting', 'actingnya', 'action', 'actionnya', 'activision', 'actor', 'actorny', 'actornya', 'actris', 'actually', 'acung', 'acungin', 'adah', 'adakan', 'adaptasi', 'adavanture', 'addicting', 'adegan', 'adek', 'adekku', 'adem', 'adick', 'adik', 'adiktif', 'adil', 'adios', 'admin', 'adminnya', 'adopsi', 'adrenalin', 'aduh', 'aduk', 'advan', 'advanture', 'adven', 'adventur', 'adventure', 'adventurenya', 'adventurnya', 'aero', 'aether', 'afeace', 'afif', 'after', 'agam', 'agat', 'agate', 'agustus', 'ah', 'ah

split datatest

In [ ]:
from sklearn.model_selection import train_test_split
label=df['score']
x_train,x_test,y_train,y_test = train_test_split(vektor_tfidf, label, test_size=0.2)
print(x_train.shape, y_train.shape)
print(x_test.shape, y_test.shape)

(3200, 5674) (3200,)
(800, 5674) (800,)


In [ ]:
print(y_test)
p = np.where(y_test == 1)
np.array(p).shape

2102    5
1246    1
2362    5
2449    5
3010    5
       ..
3230    5
1186    1
2543    5
1225    1
163     1
Name: score, Length: 800, dtype: int64


(1, 380)

pembuatan model svm

In [ ]:
from sklearn import svm
from sklearn.model_selection import cross_val_score

clf = svm.SVC(kernel="linear")
clf.fit(x_train,y_train)
y_pred = clf.predict(x_test)

In [ ]:
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
print('acc = ',accuracy_score(y_pred, y_test))
print(classification_report(y_pred, y_test))
confusion_matrix(y_pred,y_test)



acc =  0.845
              precision    recall  f1-score   support

           1       0.89      0.81      0.84       418
           5       0.81      0.89      0.85       382

    accuracy                           0.84       800
   macro avg       0.85      0.85      0.84       800
weighted avg       0.85      0.84      0.84       800



array([[337,  81],
       [ 43, 339]])